# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install --upgrade tensorflow keras Pillow

  Using cached tensorflow-2.8.0-cp38-cp38-manylinux2010_x86_64.whl (497.6 MB)
     |███████████████████▎            | 297.5 MB 113.5 MB/s eta 0:00:02

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 495.1 MB 4.2 kB/s s eta 0:00:01
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
     |████████████████████████████████| 1.3 MB 87.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 123.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 95.4 MB/s eta 0:00:01
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.2.0
    Uninstalling Pillow-8.2.0:
      Successfully uninstalled Pillow-8.2.0


In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [98]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [99]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [100]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [101]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [102]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [103]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 4.2 - Dropout + Full Connection

In [104]:
# cnn.add(tf.keras.layers.Dropout(rate=0.1))
# cnn.add(tf.keras.layers.Dense(units=128, activation='tanh'))

### Step 5 - Output Layer

In [105]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [106]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [107]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 30, batch_size=200)

Epoch 1/30
250/250 [==============================] - 15s 58ms/step - loss: 0.6532 - accuracy: 0.6145 - val_loss: 0.5828 - val_accuracy: 0.7085
Epoch 2/30
250/250 [==============================] - 15s 58ms/step - loss: 0.5691 - accuracy: 0.6996 - val_loss: 0.5506 - val_accuracy: 0.7215
Epoch 3/30
250/250 [==============================] - 15s 58ms/step - loss: 0.5367 - accuracy: 0.7296 - val_loss: 0.5008 - val_accuracy: 0.7555
Epoch 4/30
250/250 [==============================] - 15s 58ms/step - loss: 0.5022 - accuracy: 0.7525 - val_loss: 0.5477 - val_accuracy: 0.7300
Epoch 5/30
250/250 [==============================] - 14s 58ms/step - loss: 0.4855 - accuracy: 0.7604 - val_loss: 0.4910 - val_accuracy: 0.7680
Epoch 6/30
250/250 [==============================] - 15s 58ms/step - loss: 0.4633 - accuracy: 0.7836 - val_loss: 0.4999 - val_accuracy: 0.7510
Epoch 7/30
250/250 [==============================] - 15s 58ms/step - loss: 0.4451 - accuracy: 0.7930 - val_loss: 0.4466 - val_accuracy:

## Part 4 - Making a single prediction

In [17]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = cnn.predict(test_image)

if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [18]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [19]:
print(prediction)

dog


## Fim